<a href="https://colab.research.google.com/github/AneesHaider12/AneesHaider12/blob/main/aspect_based_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets pandas

In [ ]:
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12

In [ ]:
!pip install transformers sentence-transformers scikit-learn torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
from datasets import load_dataset
from datasets import disable_caching
disable_caching()
from tqdm import tqdm
import re
from transformers import AutoConfig
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

model_name = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


from datasets import load_dataset
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty")
small_dataset = dataset['full'].select(range(5000))
df = pd.DataFrame(small_dataset)


aspect_keywords = {
    "price": ["price", "cost", "expensive", "cheap", "affordable", "value", "budget", "premium", "luxury", "discount"],
    "packaging": ["packaging", "package", "container", "bottle", "jar", "tube", "design", "aesthetic", "eco-friendly", "sustainable"],
    "scent": ["scent", "smell", "fragrance", "aroma", "odor", "perfume", "unscented", "fragrance-free", "musky", "floral"],
    "ingredients": ["ingredients", "formula", "chemicals", "natural", "organic", "vegan", "cruelty-free", "preservatives", "actives", "hyaluronic acid"],
    "texture": ["texture", "consistency", "thick", "lightweight", "creamy", "oily", "sticky", "smooth", "gritty", "gel"],
    "efficacy": ["efficacy", "results", "effectiveness", "performance", "benefits", "improvement", "visible", "fast-acting", "long-lasting", "proof"],
    "brand": ["brand", "reputation", "popular", "niche", "luxury", "drugstore", "indie", "celebrity-owned", "heritage", "startup"],
    "skin_type": ["skin type", "oily", "dry", "combination", "sensitive", "acne-prone", "mature", "normal", "rosacea", "eczema"],
    "application": ["application", "blendable", "best","excellent","good","easy", "messy", "precision", "brush", "sponge", "fingers", "streaky", "patchy"],
    "sustainability": ["sustainability", "eco-friendly", "recyclable", "biodegradable", "zero-waste", "carbon-neutral", "ethical", "clean", "greenwashing", "conscious"]
}


def extract_present_keywords(sentence, aspect_keywords):
    present_terms = []
    sentence_lower = sentence.lower()
    for keywords in aspect_keywords.values():
        for keyword in keywords:
            if re.search(rf'\b{re.escape(keyword)}\b', sentence_lower):
                present_terms.append(keyword)
    return list(set(present_terms))


def predict_sentiment_by_keyword(reviews, aspect_keywords):
    labels = ["negative", "neutral", "positive"]
    all_results = []

    for review in tqdm(reviews, desc="Processing reviews"):
        matched_keywords = extract_present_keywords(review, aspect_keywords)
        if not matched_keywords:
            continue


        inputs_text = [f"[CLS] {kw} [SEP] {review} [SEP]" for kw in matched_keywords]
        inputs = tokenizer(inputs_text, return_tensors="pt", padding=True,
                           truncation=True, max_length=128).to(device)
# dont know what this part does
        with torch.no_grad():
            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=1).cpu().tolist()


        keyword_sentiments = {kw: labels[pred] for kw, pred in zip(matched_keywords, predictions)}
        all_results.append({
            "review": review,
            "term_sentiments": keyword_sentiments
        })

    return pd.DataFrame(all_results)


reviews = df['text'].tolist()
result_df = predict_sentiment_by_keyword(reviews, aspect_keywords)


result_df['term_sentiments'] = result_df['term_sentiments'].apply(lambda x: str(x))
result_df.to_csv("aspect_term_sentiments.csv", index=False)


print("Sample output:")
print(result_df.head())


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

...

The repository for McAuley-Lab/Amazon-Reviews-2023 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/McAuley-Lab/Amazon-Reviews-2023.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


All_Beauty.jsonl:   0%|          | 0.00/327M [00:00<?, ?B/s]

Generating full split: 0 examples [00:00, ? examples/s]

Processing reviews: 100%|██████████| 5000/5000 [02:52<00:00, 28.94it/s]

Sample output:
                                              review  \
0  This spray is really nice. It smells really go...   
1  This product does what I need it to do, I just...   
2                          Smells good, feels great!   
3  The polish was quiet thick and did not apply s...   
4  These were lightweight and soft but much too s...   

                                     term_sentiments  
0  {'texture': 'positive', 'good': 'positive', 'c...  
1                              {'smell': 'positive'}  
2                               {'good': 'positive'}  
3           {'dry': 'negative', 'thick': 'negative'}  
4                        {'lightweight': 'positive'}  


In [ ]:
!pip install evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=98bf69cc799a0dfbee3759eeae5b74af9ec4d8632b5ed0b840908647ffb2b5e1
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [ ]:
import os
import json
import torch
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report
from transformers import BertTokenizerFast, BertForTokenClassification, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import evaluate  # new evaluate lib instead of deprecated load_metric

# Device config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --------- Load and preprocess data ---------
df = pd.read_csv("aspect_term_sentiments.csv")
df['term_sentiments'] = df['term_sentiments'].apply(eval)

# Prepare data for Aspect Term Extraction (ATE) with token-level labels
ate_examples = []
for _, row in df.iterrows():
    review = row['review']
    tokens = review.split()
    labels = ['O'] * len(tokens)

    for aspect in row['term_sentiments'].keys():
        aspect_tokens = aspect.split()
        for i in range(len(tokens) - len(aspect_tokens) + 1):
            if tokens[i:i+len(aspect_tokens)] == aspect_tokens:
                labels[i] = 'B-ASP'
                for j in range(1, len(aspect_tokens)):
                    labels[i + j] = 'I-ASP'

    ate_examples.append({'tokens': tokens, 'labels': labels})

ate_dataset = Dataset.from_pandas(pd.DataFrame(ate_examples))

# Label encoding
label_list = ['O', 'B-ASP', 'I-ASP']
label_to_id = {l: i for i, l in enumerate(label_list)}

# --------- Tokenizer and Model for ATE ---------
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
aspect_model = BertForTokenClassification.from_pretrained(
    "bert-base-uncased", num_labels=len(label_list)
).to(device)

# Tokenize and align labels with tokens for ATE
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['tokens'],
        truncation=True,
        padding='max_length',
        is_split_into_words=True,
        max_length=128
    )
    labels = []
    for i, label in enumerate(examples['labels']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label_to_id[label[word_idx]])
            else:
                # For subwords inside the same word, use the I-label if B, else O
                if label[word_idx].startswith('B-'):
                    label_ids.append(label_to_id[label[word_idx].replace('B-', 'I-')])
                else:
                    label_ids.append(label_to_id[label[word_idx]])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_ate = ate_dataset.map(tokenize_and_align_labels, batched=True)

# Split train/test
tokenized_ate = tokenized_ate.train_test_split(test_size=0.1)
train_ate = tokenized_ate["train"]
eval_ate = tokenized_ate["test"]

# Training args for ATE
ate_args = TrainingArguments(
    output_dir="./bert-ate",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
)

# Metric for token classification using seqeval
seqeval = evaluate.load("seqeval")

def compute_token_metrics(p):
    predictions, labels = p
    preds = predictions.argmax(axis=-1)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

ate_trainer = Trainer(
    model=aspect_model,
    args=ate_args,
    train_dataset=train_ate,
    eval_dataset=eval_ate,
    tokenizer=tokenizer,
    compute_metrics=compute_token_metrics
)

# Train ATE model
ate_trainer.train()
aspect_model.save_pretrained("./finetuned-bert-ate")
tokenizer.save_pretrained("./finetuned-bert-ate")

# --------- Sentiment Classification Data Preparation ---------
sentiment_examples = []
for _, row in df.iterrows():
    review = row['review']
    for term, sentiment in row['term_sentiments'].items():
        sentiment_examples.append({
            "text": f"[CLS] {term} [SEP] {review} [SEP]",
            "label": {"negative": 0, "neutral": 1, "positive": 2}[sentiment]
        })

sentiment_dataset = Dataset.from_pandas(pd.DataFrame(sentiment_examples))

def tokenize_sentiment(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

tokenized_sentiment = sentiment_dataset.map(tokenize_sentiment, batched=True)
tokenized_sentiment.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

sentiment_data = tokenized_sentiment.train_test_split(test_size=0.1)
train_sent = sentiment_data['train']
eval_sent = sentiment_data['test']

sentiment_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3).to(device)

sentiment_args = TrainingArguments(
    output_dir="./bert-sentiment",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

def compute_sentiment_metrics(eval_preds):
    logits, labels = eval_preds
    preds = logits.argmax(-1).tolist()
    labels = labels.tolist()
    report = classification_report(labels, preds, target_names=["negative", "neutral", "positive"], output_dict=True)
    return {
        "accuracy": report["accuracy"],
        "f1_macro": report["macro avg"]["f1-score"]
    }

sentiment_trainer = Trainer(
    model=sentiment_model,
    args=sentiment_args,
    train_dataset=train_sent,
    eval_dataset=eval_sent,
    tokenizer=tokenizer,
    compute_metrics=compute_sentiment_metrics
)

# Train sentiment model
sentiment_trainer.train()
sentiment_model.save_pretrained("./finetuned-bert-sentiment")
tokenizer.save_pretrained("./finetuned-bert-sentiment")


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3261 [00:00<?, ? examples/s]

<ipython-input-14-bdb49dce8adc>:122: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  ate_trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.014220,0.902998,0.958801,0.930064,0.996356
2,No log,0.008805,0.932971,0.964419,0.948435,0.997170
3,0.028000,0.007930,0.918440,0.970037,0.943534,0.997480


Map:   0%|          | 0/8273 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-14-bdb49dce8adc>:182: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  sentiment_trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,No log,0.332442,0.893720,0.554010
2,0.383100,0.260655,0.923913,0.751887
3,0.211100,0.301766,0.923913,0.766197


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


('./finetuned-bert-sentiment/tokenizer_config.json',
 './finetuned-bert-sentiment/special_tokens_map.json',
 './finetuned-bert-sentiment/vocab.txt',
 './finetuned-bert-sentiment/added_tokens.json',
 './finetuned-bert-sentiment/tokenizer.json')

In [ ]:
import torch
from transformers import BertTokenizerFast, BertForTokenClassification, BertForSequenceClassification
import numpy as np
print(ate_model.config)
# Load fine-tuned models and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizerFast.from_pretrained("./finetuned-bert-ate")

ate_model = BertForTokenClassification.from_pretrained("./finetuned-bert-ate").to(device)
ate_model.eval()

sentiment_model = BertForSequenceClassification.from_pretrained("./finetuned-bert-sentiment").to(device)
sentiment_model.eval()

label_list = ['O', 'B-ASP', 'I-ASP']
sentiment_labels = {0: "negative", 1: "neutral", 2: "positive"}

def extract_aspects(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128).to(device)
    outputs = ate_model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=2)[0]  # shape: [seq_len]

    word_ids = inputs.word_ids(batch_index=0)  # get word mapping for tokens

    aspects = []
    current_aspect = []

    for idx, label_id in enumerate(predictions.cpu().numpy()):
        label = label_list[label_id]
        word_id = word_ids[idx]

        if word_id is None:
            continue

        # convert token id to token string
        token = tokenizer.convert_ids_to_tokens(int(inputs['input_ids'][0][idx]))

        if label == 'B-ASP':
            if current_aspect:
                aspects.append("".join(current_aspect).replace("##", ""))
                current_aspect = []
            current_aspect.append(token)
        elif label == 'I-ASP' and current_aspect:
            current_aspect.append(token)
        else:
            if current_aspect:
                aspects.append("".join(current_aspect).replace("##", ""))
                current_aspect = []

    # Add last aspect if exists
    if current_aspect:
        aspects.append("".join(current_aspect).replace("##", ""))

    # Remove duplicates and strip spaces
    clean_aspects = list(set([asp.strip() for asp in aspects if asp.strip()]))

    return clean_aspects



def classify_sentiment(aspect, review):
    # Prepare input as "[CLS] aspect [SEP] review [SEP]"
    text = f"[CLS] {aspect} [SEP] {review} [SEP]"
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128).to(device)
    outputs = sentiment_model(**inputs)
    logits = outputs.logits
    pred_label_id = torch.argmax(logits, dim=1).item()
    return sentiment_labels[pred_label_id]

# Example reviews to test
reviews = [
    "The battery life of this phone is amazing but the camera quality is poor.",
    "I love the screen resolution but the software is very buggy.",
    "The food was delicious and the service was excellent."
]

for review in reviews:
    print(f"\nReview: {review}")
    aspects =extract_aspects(review)
    print("Extracted Aspects and Sentiments:")
    for asp in aspects:
        sentiment = classify_sentiment(asp, review)
        print(f" - Aspect: {asp} | Sentiment: {sentiment}")


BertConfig {
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.52.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}


Review: The battery life of this phone is amazing but the camera quality is poor.
Extracted Aspects and Sentiments:

Review: I love the screen resolution but the software is very buggy.
Extracted As

In [ ]:
import os
import json
import torch
import pandas as pd
from sklearn.metrics import classification_report
from tqdm import tqdm
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

df = pd.read_csv("aspect_term_sentiments.csv")
df['term_sentiments'] = df['term_sentiments'].apply(eval)

# ye har review ko har aspect kay sath map krey ga aur phir ussay label krey ga
examples = []
for _, row in df.iterrows():
    review = row['review']
    for term, sentiment in row['term_sentiments'].items():
        examples.append({
            "review": review,
            "term": term,
            "text": f"[CLS] {term} [SEP] {review} [SEP]",
            "label": {"negative": 0, "neutral": 1, "positive": 2}[sentiment]
        })

train_df = pd.DataFrame(examples)
dataset = Dataset.from_pandas(train_df)


model_name = "bert-base-uncased"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3).to(device)


def preprocess(example):
    return tokenizer(example['text'], truncation=True, padding='max_length', max_length=128)

encoded_dataset = dataset.map(preprocess, batched=True)
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


data_split = encoded_dataset.train_test_split(test_size=0.1)
train_dataset = data_split["train"]
eval_dataset = data_split["test"]


training_args = TrainingArguments(
    output_dir="./bert-aspect-sentiment",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    preds = logits.argmax(-1)
    report = classification_report(labels, preds, target_names=["negative", "neutral", "positive"], output_dict=True)
    return {
        "accuracy": report["accuracy"],
        "f1_macro": report["macro avg"]["f1-score"]
    }


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


trainer.train()
trainer.evaluate()

model.save_pretrained("finetuned-bert-aspect-sentiment")
tokenizer.save_pretrained("finetuned-bert-aspect-sentiment")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/8273 [00:00<?, ? examples/s]

<ipython-input-7-a887f432599c>:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,No log,0.324964,0.891304,0.564835
2,0.374700,0.264770,0.925121,0.765034
3,0.230200,0.287221,0.919082,0.765419


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


('finetuned-bert-aspect-sentiment/tokenizer_config.json',
 'finetuned-bert-aspect-sentiment/special_tokens_map.json',
 'finetuned-bert-aspect-sentiment/vocab.txt',
 'finetuned-bert-aspect-sentiment/added_tokens.json',
 'finetuned-bert-aspect-sentiment/tokenizer.json')

In [ ]:
import re
from transformers import BertTokenizerFast, BertForSequenceClassification


model_path = "finetuned-bert-aspect-sentiment"
tokenizer = BertTokenizerFast.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path).to(device)
model.eval()

aspect_keywords = {
    "price": ["price", "cost", "expensive", "cheap", "affordable", "value", "budget", "premium", "luxury", "discount"],
    "packaging": ["packaging", "package", "container", "bottle", "jar", "tube", "design", "aesthetic", "eco-friendly", "sustainable"],
    "scent": ["scent", "smell", "fragrance", "aroma", "odor", "perfume", "unscented", "fragrance-free", "musky", "floral"],
    "ingredients": ["ingredients", "formula", "chemicals", "natural", "organic", "vegan", "cruelty-free", "preservatives", "actives", "hyaluronic acid"],
    "texture": ["texture", "consistency", "thick", "lightweight", "creamy", "oily", "sticky", "smooth", "gritty", "gel"],
    "efficacy": ["efficacy", "results", "effectiveness", "performance", "benefits", "improvement", "visible", "fast-acting", "long-lasting", "proof"],
    "brand": ["brand", "reputation", "popular", "niche", "luxury", "drugstore", "indie", "celebrity-owned", "heritage", "startup"],
    "skin_type": ["skin type", "oily", "dry", "combination", "sensitive", "acne-prone", "mature", "normal", "rosacea", "eczema"],
    "application": ["application", "blendable", "best", "excellent", "good", "easy", "messy", "precision", "brush", "sponge", "fingers", "streaky", "patchy"],
    "sustainability": ["sustainability", "eco-friendly", "recyclable", "biodegradable", "zero-waste", "carbon-neutral", "ethical", "clean", "greenwashing", "conscious"]
}


def extract_keywords(review):
    found = set()
    review_lower = review.lower()
    for category in aspect_keywords.values():
        for keyword in category:
            if re.search(rf"\b{re.escape(keyword)}\b", review_lower):
                found.add(keyword)
    return list(found)


def predict_multiple_reviews(reviews: list[str]) -> list[dict]:
    results = []

    for review in reviews:
        keywords = extract_keywords(review)
        if not keywords:
            results.append({"review": review, "term_sentiments": {}})
            continue


        inputs = [f"[CLS] {kw} [SEP] {review} [SEP]" for kw in keywords]
        encoded = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)

        with torch.no_grad():
            outputs = model(**encoded)
            preds = torch.argmax(outputs.logits, dim=1).cpu().tolist()


        term_sentiments = {kw: ["negative", "neutral", "positive"][pred] for kw, pred in zip(keywords, preds)}
        results.append({
            "review": review,
            "term_sentiments": term_sentiments
        })

    return results


sample_reviews = [
    "The scent is amazing and the texture is smooth. But it was too expensive for the quantity.",
    "I love the packaging but the product irritated my skin.",
    "Very affordable and effective. Works great for my oily skin.",
    "Terrible consistency. Felt sticky and heavy."
]

results = predict_multiple_reviews(sample_reviews)


with open("multi_review_sentiment_output.json", "w") as f:
    json.dump(results, f, indent=2)


import pprint
pprint.pprint(results)



[{'review': 'The scent is amazing and the texture is smooth. But it was too '
            'expensive for the quantity.',
  'term_sentiments': {'expensive': 'negative',
                      'scent': 'positive',
                      'smooth': 'positive',
                      'texture': 'positive'}},
 {'review': 'I love the packaging but the product irritated my skin.',
  'term_sentiments': {'packaging': 'negative'}},
 {'review': 'Very affordable and effective. Works great for my oily skin.',
  'term_sentiments': {'affordable': 'positive', 'oily': 'positive'}},
 {'review': 'Terrible consistency. Felt sticky and heavy.',
  'term_sentiments': {'consistency': 'negative', 'sticky': 'negative'}}]
